In [25]:
import pandas as pd
import numpy as np
import os
from pybedtools import BedTool


In [2]:
BASE_PATH = "/".join(os.getcwd().split("/")) # base directory level


#Wynton
BIN_PATH = os.path.join(BASE_PATH, "bin")  # where my scripts live
DATA_PATH = os.path.join(BASE_PATH, "data")  # where I dump new data 
RESULTS_PATH = os.path.join(BASE_PATH, "results")  # where I analyze results

In [3]:
# all = pd.read_csv('%s/variants_in_divergent_windows.vcf' % DATA_PATH, comment='#', sep='\t', names = ['chr','pos','id','ref','alt','qual','filter','info'])
# all = all.drop_duplicates()

# info_types = ['AC','AN','DP','AF','EAS_AF','EUR_AF','AFR_AF','AMR_AF','SAS_AF','EX_TARGET','VT','NS']
# splits=all['info'].str.split(';', expand=True)
# splits_11 = splits[splits[11].isnull()]
# splits_12 = splits[~splits[11].isnull()]
# splits_11[11] = splits_11[10]
# splits_11[10] = splits_11[9]
# splits_11[9] = 'NA'
# df = pd.concat([splits_11, splits_12], sort=False).sort_index()

# all[info_types] = df
# all = all.drop(columns=['info'])

In [4]:
# anc = pd.read_csv('%s/tree_sequences/hsmrca_vars_strip.bed' % DATA_PATH, sep='\t', names=['chr','pos','anc','red','rsid'], skiprows=1)

In [5]:
# a = pd.merge(all, anc, on=['chr','pos'], how='left')
# for col in info_types:
#     print(col)
#     if col!='EX_TARGET':
#         tmp = a[col].str.split('=', expand=True)[1]
#         if col == 'VT':
#             a[col] = tmp
#         else:
#             a[col] = tmp.astype(float)

# a.to_csv('ISM_variants_all.txt')

In [6]:

ISM_vars = pd.read_csv('ISM_variants_all.txt', index_col=0)

/scratch/egilbertson/ipykernel_39334/172544029.py:1: DtypeWarning: Columns (17,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  ISM_vars = pd.read_csv('ISM_variants_all.txt', index_col=0)


In [7]:
ISM_vars_snps = ISM_vars[ISM_vars.VT=='SNP']

In [8]:
ISM_vars_snps=ISM_vars_snps.drop(columns=['id','qual','filter','red','DP','AN','NS','EX_TARGET','VT','AC','rsid'])

In [9]:
ISM_vars_snps

,chr,pos,ref,alt,AF,EAS_AF,EUR_AF,AFR_AF,AMR_AF,SAS_AF,anc
0,1,41418882,A,T,0.01,0.00,0.0,0.03,0.0,0.00,A
1,1,41418925,A,T,0.00,0.00,0.0,0.01,0.0,0.00,A
2,1,41418953,C,G,0.00,0.00,0.0,0.00,0.0,0.00,C
3,1,41418962,G,A,0.01,0.05,0.0,0.00,0.0,0.00,G
4,1,41418972,G,A,0.00,0.00,0.0,0.00,0.0,0.00,G
...,...,...,...,...,...,...,...,...,...,...,...
9847944,22,49283013,A,T,0.00,0.00,0.0,0.00,0.0,0.00,A
9847945,22,49283015,A,G,0.00,0.00,0.0,0.00,0.0,0.00,A
9847946,22,49283054,G,C,0.00,0.00,0.0,0.00,0.0,0.00,G
9847947,22,49283057,G,C,0.00,0.00,0.0,0.00,0.0,0.00,G


In [10]:
ISM_vars_snps[ISM_vars_snps['AF'] >= 0.01]

,chr,pos,ref,alt,AF,EAS_AF,EUR_AF,AFR_AF,AMR_AF,SAS_AF,anc
0,1,41418882,A,T,0.01,0.00,0.00,0.03,0.00,0.00,A
3,1,41418962,G,A,0.01,0.05,0.00,0.00,0.00,0.00,G
14,1,41419194,G,A,0.32,0.38,0.46,0.19,0.37,0.27,G
20,1,41419448,G,A,0.02,0.01,0.03,0.00,0.02,0.02,G
26,1,41419654,C,T,0.04,0.00,0.00,0.13,0.02,0.00,C
...,...,...,...,...,...,...,...,...,...,...,...
9847939,22,49282856,A,T,0.18,0.16,0.25,0.08,0.21,0.26,A
9847940,22,49282919,T,C,0.04,0.00,0.07,0.01,0.08,0.04,NaN
9847942,22,49282994,C,T,0.02,0.00,0.00,0.08,0.01,0.00,C
9847943,22,49282999,T,A,0.85,0.99,0.80,0.89,0.69,0.84,NaN


In [11]:
ISM_vars_snps[ISM_vars_snps['AF'] >= 0.05]

,chr,pos,ref,alt,AF,EAS_AF,EUR_AF,AFR_AF,AMR_AF,SAS_AF,anc
14,1,41419194,G,A,0.32,0.38,0.46,0.19,0.37,0.27,G
28,1,41419903,T,A,0.65,0.60,0.50,0.79,0.61,0.70,T
31,1,41419982,G,A,0.62,0.60,0.50,0.68,0.59,0.71,G
34,1,41420026,C,T,0.34,0.40,0.50,0.18,0.39,0.29,T
69,1,41421235,A,G,0.05,0.01,0.03,0.11,0.03,0.02,A
...,...,...,...,...,...,...,...,...,...,...,...
9847907,22,49282246,C,A,0.07,0.00,0.07,0.14,0.09,0.04,C
9847931,22,49282787,C,T,0.17,0.16,0.25,0.02,0.21,0.26,C
9847939,22,49282856,A,T,0.18,0.16,0.25,0.08,0.21,0.26,A
9847943,22,49282999,T,A,0.85,0.99,0.80,0.89,0.69,0.84,NaN


In [12]:
ISM_vars_snps[ISM_vars_snps['AF'] >= 0.1]

,chr,pos,ref,alt,AF,EAS_AF,EUR_AF,AFR_AF,AMR_AF,SAS_AF,anc
14,1,41419194,G,A,0.32,0.38,0.46,0.19,0.37,0.27,G
28,1,41419903,T,A,0.65,0.60,0.50,0.79,0.61,0.70,T
31,1,41419982,G,A,0.62,0.60,0.50,0.68,0.59,0.71,G
34,1,41420026,C,T,0.34,0.40,0.50,0.18,0.39,0.29,T
82,1,41421729,A,G,1.00,1.00,1.00,0.99,1.00,1.00,G
...,...,...,...,...,...,...,...,...,...,...,...
9847899,22,49282160,C,T,0.29,0.47,0.27,0.11,0.20,0.40,C
9847931,22,49282787,C,T,0.17,0.16,0.25,0.02,0.21,0.26,C
9847939,22,49282856,A,T,0.18,0.16,0.25,0.08,0.21,0.26,A
9847943,22,49282999,T,A,0.85,0.99,0.80,0.89,0.69,0.84,NaN


In [22]:
def get_non_ancestral_allele(row):
    if row.anc == row.ref:
        non_anc = row.alt
        non_anc_af = row.AF
    elif row.anc == row.alt:
        non_anc = row.ref
        non_anc_af = 1-row.AF
    else:
        non_anc="??"
        non_anc_af=np.nan
    return non_anc, non_anc_af

In [23]:
ISM_vars_snps

,chr,pos,ref,alt,AF,EAS_AF,EUR_AF,AFR_AF,AMR_AF,SAS_AF,anc,non_anc,non_anc_af
0,1,41418882,A,T,0.01,0.00,0.0,0.03,0.0,0.00,A,T,0.01
1,1,41418925,A,T,0.00,0.00,0.0,0.01,0.0,0.00,A,T,0.0
2,1,41418953,C,G,0.00,0.00,0.0,0.00,0.0,0.00,C,G,0.0
3,1,41418962,G,A,0.01,0.05,0.0,0.00,0.0,0.00,G,A,0.01
4,1,41418972,G,A,0.00,0.00,0.0,0.00,0.0,0.00,G,A,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9847944,22,49283013,A,T,0.00,0.00,0.0,0.00,0.0,0.00,A,T,0.0
9847945,22,49283015,A,G,0.00,0.00,0.0,0.00,0.0,0.00,A,G,0.0
9847946,22,49283054,G,C,0.00,0.00,0.0,0.00,0.0,0.00,G,C,0.0
9847947,22,49283057,G,C,0.00,0.00,0.0,0.00,0.0,0.00,G,C,0.0


In [26]:
ISM_vars_snps[['non_anc', 'non_anc_af']] = ISM_vars_snps.apply(lambda x: get_non_ancestral_allele(x), axis=1, result_type='expand')

In [27]:
ISM_vars_snps

,chr,pos,ref,alt,AF,EAS_AF,EUR_AF,AFR_AF,AMR_AF,SAS_AF,anc,non_anc,non_anc_af
0,1,41418882,A,T,0.01,0.00,0.0,0.03,0.0,0.00,A,T,0.01
1,1,41418925,A,T,0.00,0.00,0.0,0.01,0.0,0.00,A,T,0.00
2,1,41418953,C,G,0.00,0.00,0.0,0.00,0.0,0.00,C,G,0.00
3,1,41418962,G,A,0.01,0.05,0.0,0.00,0.0,0.00,G,A,0.01
4,1,41418972,G,A,0.00,0.00,0.0,0.00,0.0,0.00,G,A,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9847944,22,49283013,A,T,0.00,0.00,0.0,0.00,0.0,0.00,A,T,0.00
9847945,22,49283015,A,G,0.00,0.00,0.0,0.00,0.0,0.00,A,G,0.00
9847946,22,49283054,G,C,0.00,0.00,0.0,0.00,0.0,0.00,G,C,0.00
9847947,22,49283057,G,C,0.00,0.00,0.0,0.00,0.0,0.00,G,C,0.00


In [28]:
ISM_vars_snps[ISM_vars_snps.non_anc_af.astype(float) >=.01]

,chr,pos,ref,alt,AF,EAS_AF,EUR_AF,AFR_AF,AMR_AF,SAS_AF,anc,non_anc,non_anc_af
0,1,41418882,A,T,0.01,0.00,0.00,0.03,0.00,0.00,A,T,0.01
3,1,41418962,G,A,0.01,0.05,0.00,0.00,0.00,0.00,G,A,0.01
9,1,41419069,C,T,0.00,0.01,0.00,0.00,0.00,0.00,T,C,1.00
14,1,41419194,G,A,0.32,0.38,0.46,0.19,0.37,0.27,G,A,0.32
20,1,41419448,G,A,0.02,0.01,0.03,0.00,0.02,0.02,G,A,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9847931,22,49282787,C,T,0.17,0.16,0.25,0.02,0.21,0.26,C,T,0.17
9847935,22,49282827,A,G,0.03,0.00,0.00,0.10,0.00,0.00,A,G,0.03
9847939,22,49282856,A,T,0.18,0.16,0.25,0.08,0.21,0.26,A,T,0.18
9847942,22,49282994,C,T,0.02,0.00,0.00,0.08,0.01,0.00,C,T,0.02


In [29]:
ISM_vars_snps.non_anc.unique()

array(['T', 'G', 'A', 'C', '??'], dtype=object)

In [30]:
ISM_vars_snps[ISM_vars_snps.non_anc=="??"]

,chr,pos,ref,alt,AF,EAS_AF,EUR_AF,AFR_AF,AMR_AF,SAS_AF,anc,non_anc,non_anc_af
11,1,41419118,G,T,0.00,0.00,0.00,0.00,0.00,0.00,NaN,??,NaN
38,1,41420139,G,A,0.00,0.01,0.00,0.00,0.00,0.00,NaN,??,NaN
45,1,41420455,C,T,0.00,0.00,0.00,0.00,0.00,0.00,NaN,??,NaN
54,1,41420829,C,T,0.04,0.05,0.04,0.03,0.02,0.05,NaN,??,NaN
99,1,41422057,C,G,0.00,0.00,0.00,0.00,0.00,0.00,NaN,??,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9847884,22,49281745,C,G,0.00,0.00,0.00,0.00,0.00,0.00,NaN,??,NaN
9847923,22,49282512,C,T,0.01,0.01,0.00,0.00,0.07,0.00,NaN,??,NaN
9847934,22,49282821,G,A,0.00,0.00,0.00,0.00,0.00,0.00,NaN,??,NaN
9847940,22,49282919,T,C,0.04,0.00,0.07,0.01,0.08,0.04,NaN,??,NaN


In [31]:
ISM_vars_snps_with_ancestral = ISM_vars_snps[ISM_vars_snps.non_anc!="??"]

In [32]:
ISM_vars_snps_with_ancestral

,chr,pos,ref,alt,AF,EAS_AF,EUR_AF,AFR_AF,AMR_AF,SAS_AF,anc,non_anc,non_anc_af
0,1,41418882,A,T,0.01,0.00,0.0,0.03,0.0,0.00,A,T,0.01
1,1,41418925,A,T,0.00,0.00,0.0,0.01,0.0,0.00,A,T,0.00
2,1,41418953,C,G,0.00,0.00,0.0,0.00,0.0,0.00,C,G,0.00
3,1,41418962,G,A,0.01,0.05,0.0,0.00,0.0,0.00,G,A,0.01
4,1,41418972,G,A,0.00,0.00,0.0,0.00,0.0,0.00,G,A,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9847944,22,49283013,A,T,0.00,0.00,0.0,0.00,0.0,0.00,A,T,0.00
9847945,22,49283015,A,G,0.00,0.00,0.0,0.00,0.0,0.00,A,G,0.00
9847946,22,49283054,G,C,0.00,0.00,0.0,0.00,0.0,0.00,G,C,0.00
9847947,22,49283057,G,C,0.00,0.00,0.0,0.00,0.0,0.00,G,C,0.00


In [33]:
ISM_vars_snps_with_ancestral_01 = ISM_vars_snps_with_ancestral[ISM_vars_snps_with_ancestral['AF'] >= 0.01]

In [35]:
ISM_vars_snps_with_ancestral_01[ISM_vars_snps_with_ancestral_01.ref != ISM_vars_snps_with_ancestral_01.anc ]

,chr,pos,ref,alt,AF,EAS_AF,EUR_AF,AFR_AF,AMR_AF,SAS_AF,anc,non_anc,non_anc_af
34,1,41420026,C,T,0.34,0.40,0.50,0.18,0.39,0.29,T,C,0.66
82,1,41421729,A,G,1.00,1.00,1.00,0.99,1.00,1.00,G,A,0.00
110,1,41422251,C,T,0.02,0.01,0.03,0.00,0.02,0.02,T,C,0.98
135,1,41422940,T,G,0.99,1.00,1.00,0.98,1.00,1.00,G,T,0.01
186,1,41424505,A,G,0.98,1.00,1.00,0.93,1.00,1.00,G,A,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9847731,22,49277621,A,G,0.88,0.99,0.81,0.99,0.70,0.80,G,A,0.12
9847782,22,49279119,A,G,0.88,0.99,0.81,0.99,0.70,0.80,G,A,0.12
9847812,22,49279925,A,G,0.97,1.00,0.97,0.93,0.97,0.99,G,A,0.03
9847835,22,49280504,G,A,0.88,0.99,0.81,0.99,0.70,0.80,A,G,0.12


In [36]:
ISM_vars_snps_with_ancestral_05 = ISM_vars_snps_with_ancestral[ISM_vars_snps_with_ancestral['AF'] >= 0.05]

In [37]:
ISM_vars_snps_with_ancestral_05

,chr,pos,ref,alt,AF,EAS_AF,EUR_AF,AFR_AF,AMR_AF,SAS_AF,anc,non_anc,non_anc_af
14,1,41419194,G,A,0.32,0.38,0.46,0.19,0.37,0.27,G,A,0.32
28,1,41419903,T,A,0.65,0.60,0.50,0.79,0.61,0.70,T,A,0.65
31,1,41419982,G,A,0.62,0.60,0.50,0.68,0.59,0.71,G,A,0.62
34,1,41420026,C,T,0.34,0.40,0.50,0.18,0.39,0.29,T,C,0.66
69,1,41421235,A,G,0.05,0.01,0.03,0.11,0.03,0.02,A,G,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9847899,22,49282160,C,T,0.29,0.47,0.27,0.11,0.20,0.40,C,T,0.29
9847907,22,49282246,C,A,0.07,0.00,0.07,0.14,0.09,0.04,C,A,0.07
9847931,22,49282787,C,T,0.17,0.16,0.25,0.02,0.21,0.26,C,T,0.17
9847939,22,49282856,A,T,0.18,0.16,0.25,0.08,0.21,0.26,A,T,0.18


In [38]:
ISM_vars_snps_with_ancestral_1 = ISM_vars_snps_with_ancestral[ISM_vars_snps_with_ancestral['AF'] >= 0.1]

In [39]:
ISM_vars_snps_with_ancestral_1

,chr,pos,ref,alt,AF,EAS_AF,EUR_AF,AFR_AF,AMR_AF,SAS_AF,anc,non_anc,non_anc_af
14,1,41419194,G,A,0.32,0.38,0.46,0.19,0.37,0.27,G,A,0.32
28,1,41419903,T,A,0.65,0.60,0.50,0.79,0.61,0.70,T,A,0.65
31,1,41419982,G,A,0.62,0.60,0.50,0.68,0.59,0.71,G,A,0.62
34,1,41420026,C,T,0.34,0.40,0.50,0.18,0.39,0.29,T,C,0.66
82,1,41421729,A,G,1.00,1.00,1.00,0.99,1.00,1.00,G,A,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9847891,22,49281890,G,A,0.29,0.47,0.27,0.12,0.20,0.41,G,A,0.29
9847899,22,49282160,C,T,0.29,0.47,0.27,0.11,0.20,0.40,C,T,0.29
9847931,22,49282787,C,T,0.17,0.16,0.25,0.02,0.21,0.26,C,T,0.17
9847939,22,49282856,A,T,0.18,0.16,0.25,0.08,0.21,0.26,A,T,0.18


In [40]:
unique_anc = ISM_vars_snps[ISM_vars_snps.non_anc=="??"][~ISM_vars_snps['anc'].isnull()]

/scratch/egilbertson/ipykernel_39334/417023092.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  unique_anc = ISM_vars_snps[ISM_vars_snps.non_anc=="??"][~ISM_vars_snps['anc'].isnull()]


In [41]:
unique_anc[unique_anc.AF >= 0.01]

,chr,pos,ref,alt,AF,EAS_AF,EUR_AF,AFR_AF,AMR_AF,SAS_AF,anc,non_anc,non_anc_af
128148,1,93336657,G,C,0.84,0.98,0.79,0.66,0.86,0.97,T,??,NaN
336761,1,160646952,A,C,0.01,0.00,0.00,0.00,0.01,0.01,G,??,NaN
704680,2,14833321,T,C,0.01,0.00,0.00,0.02,0.00,0.00,G,??,NaN
767234,2,27442464,C,T,0.01,0.00,0.00,0.03,0.00,0.00,A,??,NaN
770945,2,27594374,A,T,0.01,0.01,0.00,0.02,0.01,0.01,C,??,NaN
843919,2,35342923,T,A,0.06,0.00,0.03,0.16,0.04,0.02,C,??,NaN
1675981,2,231604730,A,G,0.03,0.01,0.05,0.02,0.04,0.03,C,??,NaN
1791317,3,38447215,C,A,0.05,0.03,0.08,0.03,0.06,0.06,G,??,NaN
1831502,3,46202793,A,C,0.03,0.00,0.00,0.11,0.01,0.00,G,??,NaN
2059895,3,129333400,T,A,0.01,0.00,0.00,0.04,0.01,0.01,C,??,NaN


In [42]:
unique_anc[unique_anc.AF >= 0.01].shape

(26, 13)